In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
## from __future__ import annotations
from pathlib import Path
from datetime import datetime
import numpy as np

In [3]:
from Chemprompt.embeddings.full_descriptors_embedding import LLMModel
from Chemprompt.data.data_loader import DataLoader
from Chemprompt.models.sklearn_model import ScikitLearnModel

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/HDD1/bbq9088/miniconda3/envs/Chemprompt/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [4]:
DATASETS = ["Caco2_Wang"]

In [5]:
MODEL_LIST = [ # {"repo": "model_repo", "name" : "model_name"}
    # {"repo": "meta-llama", "name": "Meta-Llama-3.1-8B"},
    {"repo": "CohereLabs", "name": "aya-expanse-8b"},
]

In [6]:
RESULT_ROOT = Path("./result/full_descriptors").resolve()
RESULT_ROOT.mkdir(parents=True, exist_ok=True)

In [7]:
def log(msg: str):
    print(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] {msg}")

In [8]:
def train_and_predict(x, embeds, y, save_dir: Path):
    model = ScikitLearnModel(
        model_type="regression",
        save_dir=str(save_dir),
        max_iter=5000
    )
    model.fit_and_evaluate_fold(x, embeds, y)
    # np.save(save_dir / "embeddings.npy", embeds)

In [9]:
def run_all_descriptors(dataset: str, model_info: dict):
    repo, name = model_info["repo"], model_info["name"]
    model_repo = f"{repo}/{name}"

    # data load
    loader = DataLoader()
    smiles, y = loader.load_dataset(dataset)

    # init LLM
    # llm = LLMModel(model_repo, dtype="half", device="cuda:0") #float 16
    llm = LLMModel(model_repo, dtype="full", device="cuda:0") # float 32
    llm._filter_descriptors(smiles)  # filter descriptors
    descriptor_num = len(llm.property_names)
    log(f"[INFO] Descriptor Number: {descriptor_num}.")
    
    # Initial embedding → including filtering + embedding generation
    log(f"\n[INFO] Model: {model_repo} | Dataset: {dataset}")
    embeddings = llm.get_embeddings(smiles, flag=None)
    descriptor_num = len(llm.property_names)
    log(f"[INFO] Descriptor Number: {descriptor_num}.")
    
    # save result
    save_dir = RESULT_ROOT / dataset / name
    save_dir.mkdir(parents=True, exist_ok=True)
    log(f"Training & saving to {save_dir}")
    train_and_predict(smiles, embeddings, y, save_dir)

In [10]:
if __name__ == "__main__":
    for dataset in DATASETS:
        log(f"\n=== Dataset: {dataset} ===")
        for model in MODEL_LIST:
            run_all_descriptors(dataset, model)

Found local copy...
Loading...
Done!


[2025-10-30 03:12:35] 
=== Dataset: Caco2_Wang ===
('Caco2_Wang', (910, 2))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Filtering descriptors: 100%|████████████████████████████████████████████████████████████████| 910/910 [00:13<00:00, 65.89it/s]


[2025-10-30 03:12:54] [INFO] Descriptor Number: 145.
[2025-10-30 03:12:54] 
[INFO] Model: meta-llama/Meta-Llama-3.1-8B | Dataset: Caco2_Wang


...

('Caco2_Wang', (910, 2))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Filtering descriptors: 100%|████████████████████████████████████████████████████████████████| 910/910 [00:13<00:00, 65.27it/s]


[2025-10-30 03:15:29] [INFO] Descriptor Number: 145.
[2025-10-30 03:15:29] 
[INFO] Model: CohereLabs/aya-expanse-8b | Dataset: Caco2_Wang


Embedding SMILES:  16%|███████████▎                                                         | 149/910 [00:28<02:24,  5.27it/s]

KeyboardInterrupt

